# KAFKA CONSUMER

In [1]:
from kafka import KafkaConsumer

In [8]:
consumer = KafkaConsumer('trump', bootstrap_servers=['kafka:9092'])

In [3]:
consumer.subscribe(pattern='^[Ff]rance.*')

In [29]:
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print(message.value)

b'{"created_at":"Thu Jun 24 12:54:43 +0000 2021","id":1408045925767995393,"id_str":"1408045925767995393","text":"Euro, not my thing, but sometimes it is","source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/android\\" rel=\\"nofollow\\"\\u003eTwitter for Android\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":145322639,"id_str":"145322639","name":"Hrvoje (Harvey) Hrsto","screen_name":"hrvoje_hrsto","location":"Zagreb, Cro","url":null,"description":"Problem solving water off a ducks back-er, Jonathan Van Ness lookalike, TV and audio enthusiast. 100% sci-fi natural. Tweeting in Croatian and English","translator_type":"none","protected":false,"verified":false,"followers_count":119,"friends_count":340,"listed_count":1,"favourites_count":460,"statuses_count":9670,"created_at":"Tue May 18 17:43:49 +0000 2010","utc_offset":null,"tim

KeyboardInterrupt: 

In [4]:
print(type(consumer))

<class 'kafka.consumer.group.KafkaConsumer'>


# PUT IN HDFS

In [4]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
import json

In [64]:
def convert_to_json(b):
    df_decode = b.decode('utf8')
    #df_dumps = json.dumps(df_decode)
    df_json = json.loads(df_decode)
        
    return df_json

In [70]:
inp = []

for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    inp.append(message.value)
    
    print(len(inp), end=" ")
    
    if len(inp) >= 25:
        out = map(convert_to_json, inp)
        inp = []
        break

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 

In [71]:
x1 = list(out)

In [73]:
x1

[{'created_at': 'Thu Jun 24 13:39:43 +0000 2021',
  'id': 1408057249709297667,
  'id_str': '1408057249709297667',
  'text': 'Cristiano Ronaldo 🇵🇹 doing what he knows how to do best - score great goals. \n\nHere are a few of them...\n\n🐐 \n\nhttps://t.co/wS0tIwuqO8',
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'truncated': False,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1179051625190699009,
   'id_str': '1179051625190699009',
   'name': 'Lovefootball.ng',
   'screen_name': 'lovefootballng',
   'location': 'Nigeria',
   'url': 'https://linktr.ee/Lovefootball.ng',
   'description': 'We are an online community for all football fans around the world.  Download our app to play daily fantasy contests and win cash and prizes.',
   'translator_type': 'none',
   'protected': False,
   'verified

In [74]:
from hdfs import InsecureClient
client = InsecureClient('http://hadoop-master:9870/', user='kafka-data')

In [75]:
from json import dump, dumps

# Or, passing in a generator directly:
client.write('data.json', data=dumps(x1), encoding='utf-8', overwrite=True)